In [185]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import os
from os.path import join as oj
from sklearn import metrics
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import Counter
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F
import torch
from copy import deepcopy
from sklearn import metrics
plt.style.use('dark_background')
import mat4py
import pandas as pd
import data_tracks
import models
from sklearn.model_selection import KFold
from colorama import Fore
import pickle as pkl
import viz
from style import *
import analyze_helper
import train
import data_tracks

outcome_def = 'y_consec_sig'
out_dir = 'results/outcome=y_consec_sig'
results = analyze_helper.load_results(out_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# compare all models

**look at prediction metrics**

In [ ]:
r = results
r = r[[k for k in r if not 'std' in k]]
r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
r = r.sort_values(by=['accuracy', 'balanced_accuracy'], ascending=False)
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
r.style.background_gradient(cmap='viridis', axis=0) # columns differently colored
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])

**look at feat importances**

In [ ]:
r = results
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])
r = r[r.accuracy > 0.65]
keys = [k for k in r if '_f' in k]
keys_remapped = {k: k.replace('_f', '') for k in keys}
r = r[keys].rename(columns=keys_remapped)
# r = r.sort_values('lifetime')
# r = r[r.index.str.contains('39')]
r = r[~r.index.str.contains('=11')]
# r = r[~r.index.str.contains('=7')]
r = r[~r.index.str.contains('=15')]
r = r.rename(columns={'mean_square_displacement': 'msd', 'total_displacement': 'td'})

# r = r[r.index.str.contains('11')]
# r = r[r2.index.str.contains('ros')]
# r = r[r.index.str.contains('none')]
r = r[[k for k in r if not 'std' in k]]


def rank(r):
    '''Rank feature importances appropriately
    '''
    r = r.abs()
    r = r.rank(axis=1, ascending=False, method='min')
    return r
# 
r = rank(r)
r = r.reindex(r.mean().sort_values(ascending=True).index, axis=1) # sort cols by mean rank

idxs = r.index
r.insert(0, 'acc', results.loc[idxs]['accuracy'])
r = r.sort_values('acc', ascending=False)

subset = list(r.keys())
subset.remove('acc')
r.fillna(0).style.background_gradient(cmap='viridis_r', axis=1, subset=subset) # rows differently colored

# analyze model predictions

In [188]:
df = data_tracks.get_data()
n = df.shape[0]

# normalize and store
X = df[data_tracks.get_feature_names(df)]
X_mean = X.mean()
X_std = X.std()
ks = list(X.keys())
norms = {ks[i]: {'mu': X_mean[i], 'std': X_std[i]} for i in range(len(ks))}
X = (X - X_mean) / X_std
y = df[outcome_def].values


# split testing data based on cell num
cv_idx = data_tracks.cell_nums_train[0]
idxs_test = df.cell_num.isin([cv_idx]) # this is the cv set for model's trained in the first cv fold
X_test, Y_test = X[idxs_test], y[idxs_test]

# look at best models
r = results
r = r.sort_values('accuracy', ascending=False)
idx = np.array(r.index)
print(idx[:10])

['mlp2_15_ros_select_rf=3' 'svm_25_ros_select_rf=11'
 'svm_25_ros_select_rf=5' 'mlp2_45_ros_select_rf=7'
 'svm_15_ros_select_rf=5' 'mlp2_25_ros_select_rf=7'
 'mlp2_25_ros_select_rf=3' 'svm_25_ros_select_rf=3'
 'svm_45_ros_select_rf=5' 'svm_15_ros_select_rf=11']


### look at single model

In [195]:
model_names = idx[:12]
model_name = idx[0]

In [196]:
d_full_cv, y_full_cv = analyze_helper.get_data_over_folds(model_name, out_dir, df.cell_num, X, y)
preds, preds_proba = d_full_cv[model_name], d_full_cv[model_name + '_proba']

results_individual = pkl.load(open(oj(out_dir, f'{model_name}.pkl'), 'rb'))
assert np.mean(preds==y_full_cv) == np.average(results_individual['cv']['accuracy'], weights=results_individual['num_pts_by_fold_cv']), 'did not properly load model/data'

In [197]:
# model_name = 'dt_15_none_select_rf=7'
results_individual = pkl.load(open(oj(out_dir, f'{model_name}.pkl'), 'rb'))
preds, preds_proba = analyze_helper.analyze_individual_results(results_individual, X_test, Y_test, 
                                                print_results=False, plot_results=False)

In [ ]:
viz.viz_errs_lifetime(X_test, preds, preds_proba, Y_test, norms)

In [ ]:
viz.viz_errs_spatially(df, idxs_test, preds, Y_test)

In [ ]:
viz.viz_errs_outliers(X_test, preds, Y_test, num_feats_reduced=10)

In [ ]:
idxs_fp = preds > Y_test
idxs_fn = preds < Y_test
print('fp')
viz.viz_biggest_errs(df['X'][idxs_test][idxs_fp], Y_test[idxs_fp], preds[idxs_fp], preds_proba[idxs_fp])    
print('fn')
viz.viz_biggest_errs(df['X'][idxs_test][idxs_fn], Y_test[idxs_fn], preds[idxs_fn], preds_proba[idxs_fn])    

### look at many models

In [101]:
# model_names = ['mlp2_11_none', 'svm_35_none', 'logistic_4_none', 'rf_9_none']
# model_names = ['mlp2_11_none', 'mlp2_9_none', 'mlp2_4_none']
df_preds = analyze_helper.load_results_many_models(out_dir, model_names, X_test, Y_test)

**venn-diagram**

In [ ]:
sets = []
for model_name in model_names[:3]:
    args = np.argwhere(df_preds[model_name + '_errs'])
    sets.append(set(args.flatten().tolist()))
    
plt.figure(dpi=300)
plt.title('venn diagram of shared errors')
venn3(sets, model_names[:3])
plt.show()

# ensemble err

**quick check**

In [ ]:
d_full_cv, y_full_cv = analyze_helper.get_data_over_folds(model_names[0:3], out_dir, df.cell_num, X, y)

In [130]:
# ensemble
preds_soft = d_full_cv.sum(axis=1) / d_full_cv.shape[1]

for score_name in train.scorers.keys():
    print(score_name, train.scorers[score_name](y_full_cv, preds_soft > 0.5))
# metrics.accuracy_score(y_ensemble, preds_soft>0.5) 

balanced_accuracy 0.6775520859231267
accuracy 0.6921587608906099
precision 0.7877813504823151
recall 0.7248520710059172
f1 0.7550077041602465
roc_auc 0.6775520859231268
precision_recall_curve (array([0.65440465, 0.78778135, 1.        ]), array([1.        , 0.72485207, 0.        ]), array([False,  True]))
roc_curve (array([0.       , 0.3697479, 1.       ]), array([0.        , 0.72485207, 1.        ]), array([2, 1, 0]))
